In [1]:
# Import libraries
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# NLTK VADER for sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
from transformers import pipeline
import numpy as np

In [2]:
df = pd.read_csv("train.csv")
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
neutral = []
for i in df.values:
    if i[2] == 0 and i[3] == 0 and i[4] == 0 and i[5] == 0 and i[6] == 0 and i[6] == 0:
        neutral.append(0)
    else:
        neutral.append(1)

In [4]:
neutral = np.array(neutral)
df['neutral'] = neutral
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,neutral
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0


In [5]:
from collections import Counter

Counter(neutral)

Counter({0: 143400, 1: 16171})

In [6]:
df_neu = df[df.neutral == 0]
df_neu.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,neutral
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0


In [7]:
text = df_neu['comment_text'].values
text[1]

"D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"

In [8]:
analyser = SentimentIntensityAnalyzer()

scores = df_neu['comment_text'].apply(analyser.polarity_scores).tolist()
ids = df_neu.index.values.tolist()
# print(len(scores))
count = 1
for i in scores:
#     print("count" + str(count) + str(i))
    count+= 1
scores_df = pd.DataFrame(scores)
scores_df['id'] = ids
scores_df.set_index('id', inplace=True)
scores_df.head()

,neg,neu,pos,compound
id,,,,
0,0.000,0.897,0.103,0.5574
1,0.099,0.743,0.158,0.2942
2,0.083,0.849,0.068,-0.1779
3,0.022,0.916,0.062,0.5106
4,0.000,0.663,0.337,0.6808


In [9]:
df_neu = pd.concat([df_neu,scores_df],axis=1)
df_neu.head(20)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,neutral,neg,neu,pos,compound
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0,0.000,0.897,0.103,0.5574
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0,0.099,0.743,0.158,0.2942
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0,0.083,0.849,0.068,-0.1779
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0,0.022,0.916,0.062,0.5106
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0,0.000,0.663,0.337,0.6808
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,0,0.000,0.464,0.536,0.7964
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,0,0.129,0.773,0.099,-0.1779
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,0,0.109,0.891,0.000,-0.8020
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0,0,0.000,1.000,0.000,0.0000
10,0005300084f90edc,"""\nFair use rationale for Image:Wonju.jpg\n\nT...",0,0,0,0,0,0,0,0.019,0.877,0.104,0.9910


In [10]:
text = df_neu['comment_text'].values
text[6]

"Your vandalism to the Matt Shirvington article has been reverted.  Please don't do it again, or you will be banned."

In [11]:
df_sub = df_neu[df_neu.neu < 0.75]
df_sub.head(20)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,neutral,neg,neu,pos,compound
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0,0.099,0.743,0.158,0.2942
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0,0.000,0.663,0.337,0.6808
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,0,0.000,0.464,0.536,0.7964
14,00070ef96486d6f9,"Oh, and the girl above started her arguments w...",0,0,0,0,0,0,0,0.160,0.668,0.171,0.1280
20,000b08c464718505,"""\n\n Regarding your recent edits \n\nOnce aga...",0,0,0,0,0,0,0,0.230,0.674,0.096,-0.7741
21,000bfd0867774845,"""\nGood to know. About me, yeah, I'm studying ...",0,0,0,0,0,0,0,0.000,0.579,0.421,0.6249
30,0011cc71398479c4,How could I post before the block expires? Th...,0,0,0,0,0,0,0,0.146,0.707,0.146,0.0000
31,00128363e367d703,Not sure about a heading of 'Fight for Freedom...,0,0,0,0,0,0,0,0.257,0.507,0.236,0.1625
32,001325b8b20ea8aa,Praise \n\nlooked at this article about 6 mont...,0,0,0,0,0,0,0,0.000,0.544,0.456,0.7717
38,00169857adbc989b,"Hi Explicit, can you block O Fenian for edit-w...",0,0,0,0,0,0,0,0.238,0.698,0.063,-0.7579


In [12]:
text = df_sub['comment_text'].values
text[10]

"Are you threatening me for disputing neutrality? I know in your country it's quite common to bully your way through a discussion and push outcomes you want. But this is not Russia."

In [13]:
sub_list = df_sub['id'].values

In [14]:
sub_list = sub_list.tolist()

In [15]:
df.loc[df['id'].isin(sub_list), 'neutral'] = 1

In [16]:
df.head(20)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,neutral
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,1
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,1
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,1
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,1
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0,0


In [18]:
text = df['comment_text'].values
print(text[2])
print('\n')
print(text[3])
print('\n')
print(text[7])
print('\n')
print(text[8])

Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.


"
More
I can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.

There appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport  "


Your vandalism to the Matt Shirvington article has been reverted.  Please don't do it again, or you will be banned.


Sorry if the word 'no

In [37]:
df.to_csv('train_with_subjectivity_labels.csv',index = False)